In [ ]:
import dash
import psycopg2
try:
    connection = psycopg2.connect(
        host='localhost',
        user='postgres', # Usuario de la base de datos (Cambiar si es necesario)
        password='2I@@`7ao*3}*', # Contraseña de la base de datos (Cambiar si es necesario)
        database='proyecto'
    )
    print("Conexión exitosa")
    cursor = connection.cursor()
# Manejo de errores
except Exception as error:
    print("Error al conectar a la base de datos", error)
# Cerrar la conexión
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Conexión cerrada")

In [ ]:
# Asignación de las tablas a variables
t_Albumes = "select * from Albumes"
t_Canciones = "select * from Canciones"
t_Artistas = "select * from Artistas"
t_Cancion_Artista = "select * from Cancion_Artista"
t_Paises = "select * from Paises"
t_Snapshots = "select * from Snapshots"
t_Rankings = "select * from Rankings"

In [ ]:
#Asignación de las consultas a variables

# Consulta 1: Relación entre atributos y cambios en el ranking
t_consulta1 = "SELECT c.nombre AS cancion, r.pais_codigo, r.ranking_diario, r.movimiento_diario, r.movimiento_semanal, c.danceability, c.energy, c.tempo FROM Rankings r JOIN Canciones c ON r.spotify_id = c.spotify_id WHERE r.movimiento_diario IS NOT NULL AND r.movimiento_diario > 0 OR r.movimiento_semanal IS NOT NULL AND r.movimiento_semanal > 0 ORDER BY r.movimiento_diario DESC, r.movimiento_semanal DESC;"
# Consulta 2: Canciones con más de 30 días en el ranking
t_consulta2 = "SELECT c.nombre AS cancion, p.nombre AS pais, COUNT(r.fecha_snapshot) AS dias_en_ranking, AVG(r.ranking_diario) AS promedio_ranking FROM Rankings r JOIN Canciones c ON r.spotify_id = c.spotify_id JOIN Paises p ON r.pais_codigo = p.pais_codigo GROUP BY c.nombre, p.nombre HAVING COUNT(r.fecha_snapshot) > 30 ORDER BY dias_en_ranking DESC, promedio_ranking ASC;"
# Consulta 3: Popularidad de canciones por país y análisis de atributos
t_consulta3 = "SELECT p.nombre AS pais, c.nombre AS cancion, AVG(r.ranking_diario) AS promedio_ranking, MAX(r.movimiento_diario) AS max_movimiento_diario, MAX(r.movimiento_semanal) AS max_movimiento_semanal, AVG(c.danceability) AS promedio_danceability, AVG(c.energy) AS promedio_energy, AVG(c.tempo) AS promedio_tempo FROM Rankings r JOIN Canciones c ON r.spotify_id = c.spotify_id JOIN Paises p ON r.pais_codigo = p.pais_codigo GROUP BY p.nombre, c.nombre ORDER BY promedio_ranking ASC, max_movimiento_diario DESC;"
# Consulta 4: Factores que afectan la longevidad en el ranking según atributos.
t_consulta4 = "SELECT c.nombre AS cancion, p.nombre AS pais, AVG(c.danceability) AS promedio_danceability, AVG(c.energy) AS promedio_energy, AVG(c.tempo) AS promedio_tempo, COUNT(r.fecha_snapshot) AS dias_en_ranking FROM Rankings r JOIN Canciones c ON r.spotify_id = c.spotify_id JOIN Paises p ON r.pais_codigo = p.pais_codigo GROUP BY c.nombre, p.nombre HAVING COUNT(r.fecha_snapshot) > 15 ORDER BY dias_en_ranking DESC;"

In [ ]:
# Tabla Albumes
cursor = connection.cursor()
cursor.execute(t_Albumes)
rows = cursor.fetchall()
print('TABLA ALBUMES: album_id, nombre, fecha_lanzamiento')
for row in rows:
    print(row)
print("\n")

In [ ]:
# Tabla Canciones
cursor = connection.cursor()
cursor.execute(t_Canciones)
rows = cursor.fetchall()
print('TABLA CANCIONES: spotify_id, nombre, es_explicito, duracion_ms, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature, album_id')
for row in rows:
    print(row)
print("\n")

In [ ]:
# Tabla Artistas
cursor = connection.cursor()
cursor.execute(t_Artistas)
rows = cursor.fetchall()
print('TABLA ARTISTAS: artista_id, nombre')
for row in rows:
    print(row)
print("\n")

In [ ]:
# Tabla Cancion_Artista
cursor = connection.cursor()
cursor.execute(t_Cancion_Artista)
rows = cursor.fetchall()
print('TABLA CANCION_ARTISTA: spotify_id, artista_id')
for row in rows:
    print(row)
print("\n")

In [ ]:
# Tabla Paises
cursor = connection.cursor()
cursor.execute(t_Paises)
rows = cursor.fetchall()
print('TABLA PAISES: pais_codigo, nombre')
for row in rows:
    print(row)
print("\n")

In [ ]:
 # Tabla Snapshots
cursor = connection.cursor()
cursor.execute(t_Snapshots)
rows = cursor.fetchall()
print('TABLA SNAPSHOTS: fecha_snapshot')
for row in rows:
    print(row)
print("\n")

In [ ]:
 # Tabla Rankings
cursor = connection.cursor()
cursor.execute(t_Rankings)
rows = cursor.fetchall()
print('TABLA RANKINGS: spotify_id, pais_codigo, fecha_snapshot, ranking_diario, movimiento_diario, movimiento_semanal')
for row in rows:
    print(row)
print("\n")

In [ ]:
 # Consulta 1
cursor = connection.cursor()
cursor.execute(t_consulta1)
rows = cursor.fetchall()
print('CONSULTA 1: cancion, pais_codigo, ranking_diario, movimiento_diario, movimiento_semanal, danceability, energy, tempo')
for row in rows:
    print(row)
print("\n")

In [ ]:
 # Consulta 2
cursor = connection.cursor()
cursor.execute(t_consulta2)
rows = cursor.fetchall()
print('CONSULTA 2: cancion, pais, dias_en_ranking, promedio_ranking')
for row in rows:
    print(row)
print("\n")

In [ ]:
# Consulta 3
cursor = connection.cursor()
cursor.execute(t_consulta3)
rows = cursor.fetchall()
print('CONSULTA 3: pais, cancion, promedio_ranking, max_movimiento_diario, max_movimiento_semanal, promedio_danceability, promedio_energy, promedio_tempo')
for row in rows:
    print(row)
print("\n")

In [ ]:
# Consulta 4
cursor = connection.cursor()
cursor.execute(t_consulta4)
rows = cursor.fetchall()
print('CONSULTA 4: cancion, pais, promedio_danceability, promedio_energy, promedio_tempo, dias_en_ranking')
for row in rows:
    print(row)
print("\n")

In [ ]:
#Gráficas
from dash import Dash, html, dcc
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px    


In [ ]:
#Gráfica de pie y barras de la consulta 1

cursor=connection.cursor()    
cursor.execute(t_consulta1)
rows=cursor.fetchall()
for row in rows:
    print(row)
    
app = Dash(__name__)  

fig1 = px.pie(rows, values=1, names=0, color_discrete_sequence=["#b52a64"])


cursor=connection.cursor()    
cursor.execute(t_consulta1)
rows=cursor.fetchall()
for row in rows:
    print(row)
    
app = Dash(__name__)  

fig2 = px.bar(rows, values=1, names=0, color_discrete_sequence=["#b52a64"])   

In [ ]:
#Gráfica lineal y de calor de la consulta 2

cursor=connection.cursor()    
cursor.execute(t_consulta2)
rows=cursor.fetchall()
for row in rows:
    print(row)
    
app = Dash(__name__)  

fig3 = px.line(rows, values=1, names=0, color_discrete_sequence=["#b52a64"])


cursor=connection.cursor()    
cursor.execute(t_consulta2)
rows=cursor.fetchall()
for row in rows:
    print(row)
    
app = Dash(__name__)  

fig4 = px.sunburst(rows, values=1, names=0, color_discrete_sequence=["#b52a64"])   

In [ ]:
#Gráfica de barras y de dispersión de la consulta 3

cursor=connection.cursor()
cursor.execute(t_consulta3)
rows=cursor.fetchall()
for row in rows:
    print(row)

app = Dash(__name__)

fig5 = px.bar(rows, x=0, y=1, color=2, barmode='group')

cursor=connection.cursor()
cursor.execute(t_consulta3)
rows=cursor.fetchall()
for row in rows:
    print(row)
    
app = Dash(__name__)

fig6 = px.scatter(rows, x=0, y=1, color=2)

In [ ]:
#Gráfica de pie y de caja de la consulta 4

cursor=connection.cursor()
cursor.execute(t_consulta4)
rows=cursor.fetchall()
for row in rows:
    print(row)

app = Dash(__name__)

fig7 = px.pie(rows, values=1, names=0, color_discrete_sequence=["#b52a64"])

cursor=connection.cursor()
cursor.execute(t_consulta4)
rows=cursor.fetchall()
for row in rows:
    print(row)

app = Dash(__name__)

fig8 = px.box(rows, x=0, y=1, color=2)

In [ ]:
app.layout = html.Div(children=[
    html.H1(children='Gráficas de Consultas'),
    html.H2(children='Consulta 1'),
    dcc.Graph(figure=fig1),
    dcc.Graph(figure=fig2),
    html.H2(children='Consulta 2'),
    dcc.Graph(figure=fig3),
    dcc.Graph(figure=fig4),
    html.H2(children='Consulta 3'),
    dcc.Graph(figure=fig5),
    dcc.Graph(figure=fig6),
    html.H2(children='Consulta 4'),
    dcc.Graph(figure=fig7),
    dcc.Graph(figure=fig8)
])